# Processing predictions made by TIMES

- Created by: Louis Groff
- PIs: Imran Shah and Grace Patlewicz (GP)
- Last modified by GP: 28 March 2024
- Changes made: Additional notes captured on how TIMES was used as taken from the SI. Metsim function moved to the metsim package and an example test chemical run to showcase the functionality.



## Running TIMES

Launch TIMES and import text (saved as txt) containing CASRN, Name and SMILES as headers. Load file and save as odb file locally. Open either In Vivo Rat Simulator or In Vitro Rat Liver S9 metabolism opptions. Within the tab corresponding to the simulator, input parameters can be accessed via the “Metabolization” button within the Model Options toolbar. In the manuscript, both metabolism simulators were set to the same Max Level of 6, with the probability of obtaining a metabolite (P. Obtain) thresholded at 0.1. For both simulators, the “Max count” parameter that thresholds the maximum number of metabolites per cycle was increased to 10 per cycle. All other parameters were left at their default settings. It is worth noting as well that increasing the “Rival Pathways” parameter to greater than one will typically increase the breadth of a transformation tree out to the selected transformation depth, and that often, transformation depth is maximised at 5 to 6 cycles at most, given phase II elimination rules that terminate transformation pathways within TIMES.

The metsim_run_times function takes as
input: CSV output from TIMES (current version 2.31.2.82 as of 08/2022, with first column label changed from "#" to "Parent Number"), model name.
    action: Takes existing CSV output from TIMES, and converts the CSV input dataframe to standardised dictionary output with appropriate generational tracking. Queries Cheminformatics Modules and Comptox Chemicals Dashboard for parent/metabolite metadata.
    output: Standardised dictionary of parent-metabolite relationships with their relevant metadata. CI Modules/CCD queries kept in this restructuring function since all metabolism simulation is completed manually in the TIMES GUI before input into this function.

### Import relevant libraries including the metsim functions

In [1]:
import os, sys
import pandas as pd

In [2]:
TOP = os.getcwd().replace("notebooks", "")
raw_dir = TOP + 'data/raw/'
processed_dir = TOP + 'data/processed/'
figures_dir = TOP + 'reports/figures/'
jcim = TOP + 'data/raw/JCIM_PhaseI/'
smpdb = TOP + 'data/raw/smpdb_drugs/'
missing = TOP + 'data/raw/extra_metsim/'

In [3]:
LIB = os.getcwd().replace("notebooks", "")

In [4]:
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

In [5]:
from metsim.sim.metsim_times import *

### Read in the TIMES output from the 112 dataset as a df. Limit the output to the first parent chemical for the purposes of the test example

In [6]:
df = pd.read_csv(processed_dir+'times_metsim_ratlivers9_6gens_10mtbs_rivalsto6_112parents.csv')

In [7]:
df1 = df[df['Parent Number'] ==1]

### View the dataframe for the first parent chemical and its simulated metabolites

In [10]:
df1

,Parent Number,ID of metabolite,Level of generation,Predecessor ID,CAS,Chemical name,SMILES,Observed map/metabolite,Transformation name,Transformation No,Transformation ID,Transformation probability group,Transformation expert group,Transformation type,Transformation probability,Used probability,Quantity of metabolite,Total quantity of metabolite,Number of repetitions
0,1,1,0,-,103-90-2,DTXSID2020006,CC(=O)Nc1ccc(O)cc1,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0280,0.0280,1
1,1,2,1,1,NaN,NaN,CC(=O)N=C1C=CC(=O)C=C1,Yes,Quinone Imine Formation|4-Aminophenol Quinone ...,192.0,235.0,0.0,34.0,[phase I],0.95,0.6368,0.2184,0.2184,1
2,1,3,2,2,NaN,NaN,CC(=O)Nc1ccc(O)cc1SCC(C(=O)NCC(O)=O)NC(=O)CCC(...,No,Glutathione Conjugation|Quinone Imine Glutathi...,508.0,236.0,66.0,66.0,[phase II],0.30,0.2190,0.1395,0.1395,1
3,1,4,2,2,NaN,NaN,CC(=O)N=C1CC(SCC(C(=O)NCC(O)=O)NC(=O)CCC(N)C(O...,No,"Glutathione Conjugation|Alpha,beta Conjugated ...",515.0,404.0,66.0,66.0,[phase II],0.30,0.2190,0.1395,0.2789,2
4,1,5,2,2,NaN,NaN,CC(=O)N=C1CC(SCC(C(=O)NCC(O)=O)NC(=O)CCC(N)C(O...,No,"Glutathione Conjugation|Alpha,beta Conjugated ...",515.0,404.0,66.0,66.0,[phase II],0.30,0.2190,0.1395,0.2789,2
5,1,6,1,1,NaN,NaN,CC(=O)Nc1ccc(OC2C(O)C(O)C(O)C(C(O)=O)O2)cc1,Yes,"O-Glucuronidation|Alcohol, Phenol, Carboxylic ...",506.0,222.0,0.0,2.0,[phase II],0.30,0.2011,0.2011,0.2011,1
6,1,7,1,1,NaN,NaN,CC(=O)Nc1ccc(OS(O)(=O)=O)cc1,Yes,Sulfation|Phenol Sulfation,537.0,296.0,0.0,19.0,[phase II],0.20,0.1341,0.1341,0.1341,1


### Apply the metsim_run_time to the output for the first parent chemical captured in the TIMES export

In [8]:
test_run = metsim_run_times(df1)

Attempting query of Cheminformatics Modules Standardizer with SMILES: CC(=O)Nc1ccc(O)cc1...
Query succeeded.
Input query added to metadata cache...
Attempting query of Cheminformatics Modules Standardizer with SMILES: CC(=O)N=C1C=CC(=O)C=C1...
Query succeeded.
successor metabolite query added to metadata cache...
Attempting query of Cheminformatics Modules Standardizer with SMILES: CC(=O)Nc1ccc(OC2C(O)C(O)C(O)C(C(O)=O)O2)cc1...
Query succeeded.
successor metabolite query added to metadata cache...
Attempting query of Cheminformatics Modules Standardizer with SMILES: CC(=O)Nc1ccc(OS(O)(=O)=O)cc1...
Query succeeded.
successor metabolite query added to metadata cache...
Precursor SMILES found in cached results. Inserting into dictionary...
Attempting query of Cheminformatics Modules Standardizer with SMILES: CC(=O)Nc1ccc(O)cc1SCC(C(=O)NCC(O)=O)NC(=O)CCC(N)C(O)=O...
Query succeeded.
successor metabolite query added to metadata cache...
Precursor SMILES found in cached results. Inserting in

### Print out the output of the results to show the json format from the TIMES processing using metsim_run_times

In [9]:
test_run

[{'datetime': '2024-03-28_12h40m13s',
  'software': 'OASIS TIMES',
  'version': '2.31.2.82',
  'params': {'depth': 3,
   'organism': 'Rat',
   'site_of_metabolism': False,
   'model': ['In Vitro Rat Liver S9 v12.18']},
  'input': {'smiles': 'CC(=O)Nc1ccc(O)cc1',
   'casrn': '103-90-2',
   'hcd_smiles': 'CC(NC1C=CC(O)=CC=1)=O',
   'inchikey': 'RZVAJINKPMORJF-BGGKNDAXNA-N',
   'dtxsid': 'DTXSID2020006',
   'chem_name': 'Acetaminophen',
   'likelihood': None},
  'output': [{'precursor': {'smiles': 'CC(=O)Nc1ccc(O)cc1',
     'casrn': '103-90-2',
     'hcd_smiles': 'CC(NC1C=CC(O)=CC=1)=O',
     'inchikey': 'RZVAJINKPMORJF-BGGKNDAXNA-N',
     'dtxsid': 'DTXSID2020006',
     'chem_name': 'Acetaminophen',
     'likelihood': None},
    'successors': [{'enzyme': '[phase I]',
      'mechanism': 'Quinone Imine Formation|4-Aminophenol Quinone Imine Formation',
      'metabolite': {'smiles': 'CC(=O)N=C1C=CC(=O)C=C1',
       'casrn': '50700-49-7',
       'hcd_smiles': 'CC(N=C1C=CC(=O)C=C1)=O',
      